In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import *

In [2]:
ND=64  #1辺の差分分割数

In [3]:
# 初期濃度場の生成
def init():
    for i in range(ND):
        for j in range(ND):
            ch[i][j]=c0+0.01*(2.0*rand()-1.0)
    return ch

In [4]:
# 計算条件における定数の設定
ndm=ND-1

ch=[[0.0 for i in range(ND)] for j in range(ND)]
ch2=[[0.0 for i in range(ND)] for j in range(ND)]
ck=[[0.0 for i in range(ND)] for j in range(ND)]

c0=0.4 # 合金組成（B成分のモル分率）
al=60.0*1.e-9 # 二次元計算領域一辺の長さ[m]
b1=al/ND # 差分ブロック一辺の長さ[m]

time1=0 # 初期時間（実時間ではなく繰り返し回数）
time1max=10000 # 終了時間（実時間ではなく繰り返し回数）
delt=0.01 # 時間きざみ[無次元]

rr=8.3145 # ガス定数[J／（mol K）]
temp=1000. # 時効温度[K]
rtemp=rr*temp # エネルギーの規格化定数RT[J/mol]

L0=25000.0/rtemp # 原子間相互作用パラメータ[J/mol]/RT
kappa_c=5.0e-15/b1/b1/rtemp # 濃度勾配エネルギー係数[Jm^2/mol]/b1^2/RT

Mc=c0*(1.0-c0) # 原子拡散の易動度[無次元]

In [5]:
# 初期濃度場の設定
ch=init()

In [ ]:
# 濃度場の時間発展の計算ループ
for time1 in range(time1max):

#******[ポテンシャルの計算]**************************************
    for i in range(ND):
        for j in range(ND):
            ip=i+1 #周期的境界条件
            im=i-1
            jp=j+1
            jm=j-1
            if i==ndm:
                ip=0
            if i==0:
                im=ndm  
            if j==ndm:
                jp=0
            if j==0:
               jm=ndm

            mu_chem=L0*(1.0-2.0*ch[i][j])+np.log(ch[i][j])-np.log(1.-ch[i][j]) #化学ポテンシャルの差
            mu_surf=-2.0*kappa_c*(ch[ip][j]+ch[im][j]+ch[i][jp]+ch[i][jm]-4.0*ch[i][j]) #濃度勾配エネルギーポテンシャル
            ck[i][j]=mu_chem+mu_surf #拡散ポテンシャル

#******[発展方程式の計算]**************************************
    for i in range(ND):
        for j in range(ND):
            ip=i+1 #周期的境界条件
            im=i-1
            jp=j+1
            jm=j-1
            if i==ndm:
                ip=0
            if i==0:
                im=ndm  
            if j==ndm:
                jp=0
            if j==0:
               jm=ndm
            cddtt=Mc*(ck[ip][j]+ck[im][j]+ck[i][jp]+ck[i][jm]-4.0*ck[i][j]) #非線形拡散方程式 式(6-12)
            ch2[i][j]=ch[i][j] + cddtt * delt #陽解法


    sumc=0.0 #濃度場の収支補正
    for i in range(ND):
        for j in range(ND):
            sumc+=ch2[i][j]
    dc0=sumc/ND/ND-c0
    for i in range(ND):
        for j in range(ND):
            ch[i][j]=ch2[i][j]-dc0
            if ch[i][j]>=1.0: # 濃度変域の補正
                ch[i][j]=0.99999
            if ch[i][j]<=0.0:
                ch[i][j]=0.00001

# 描画
    if time1 % 200 == 0:  # 200タイムステップごとに描画
        plt.clf()  # 描画内容クリア
        plt.imshow(ch)  # 濃度場の描画
        plt.clim(0, 1)  # カラースケールの最大、最小の設定
        plt.pause(0.01) # 0.01秒間表示
